In [147]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [148]:
# Year of part B dataset, must be adjusted to have different results*
year_of_df = "2021"

In [149]:
# Define the columns you want to read
df_features = ['Rndrng_NPI', 'Rndrng_Prvdr_Type', 'Rndrng_Prvdr_Gndr', 'Tot_Benes', 
               'Tot_Srvcs', 'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Pymt_Amt']

# Columns you want to one-hot encode
columns_to_encode = ['Rndrng_Prvdr_Type', 'Rndrng_Prvdr_Gndr']

# Define the base path or pattern of the file names
base_path = 'Medicare_Physician_Other_Practitioners_by_Provider_and_Service_{}.csv'

# Loop through each year, format the filename, read the CSV with only the specified columns
file_path = base_path.format(year_of_df)
df_final = pd.read_csv(file_path, usecols=df_features, low_memory=False, encoding='ISO-8859-1')
    
# Drop rows where 'Rndrng_Prvdr_Gndr' is NaN
df_final.dropna(subset=['Rndrng_Prvdr_Gndr'], inplace=True)
    
# Perform one-hot encoding on the specified columns
#     df = pd.get_dummies(df, columns=columns_to_encode)

In [150]:
df_leie = pd.read_csv('LEIE_2024.csv', low_memory=False)

df_leie['EXCLDATE'] = df_leie['EXCLDATE'].astype(str).str.pad(8, fillchar='0')

# Extract the year and month from EXCLDATE
df_leie['Year'] = df_leie['EXCLDATE'].str.slice(0, 4).astype(int)
df_leie['Month'] = df_leie['EXCLDATE'].str.slice(4, 6).astype(int)

# Determine the nearest year
df_leie['Nearest Year'] = df_leie.apply(lambda row: row['Year'] + 1 if row['Month'] >= 7 else row['Year'], axis=1)
df_leie['Nearest Year'].unique()

df_leie['fraud'] = (df_leie['Nearest Year'] >= int(year_of_df)).astype(int)
df_leie = df_leie.loc[:, ['NPI', 'fraud', 'Nearest Year']]
df_leie = df_leie[df_leie['NPI'] != 0]

df_leie = df_leie.drop(columns=['Nearest Year'])
df_leie[df_leie['fraud'] == 0]
df_leie[df_leie['fraud'] == 1]

array([2020, 2023, 2022, 1989, 1997, 2018, 2009, 1994, 2003, 2012, 2004,
       1998, 1992, 1987, 2019, 1993, 2002, 1996, 2000, 2017, 2007, 1991,
       2013, 2005, 2001, 1995, 1990, 2008, 2016, 1999, 2006, 2015, 2014,
       1988, 2024, 2021, 1980, 2010, 1979, 1986, 2011, 1981, 1984, 1985,
       1982, 1978, 1983], dtype=int64)

,NPI,fraud
6,1922348218,0
26,1942476080,0
30,1275600959,0
33,1891731758,0
65,1851631543,0
...,...,...
79296,1740241843,0
79322,1124024435,0
79329,1194930149,0
79339,1578637385,0


,NPI,fraud
2,1972902351,1
43,1265830335,1
67,1073916631,1
80,1437510278,1
119,1801231436,1
...,...,...
79232,1710929609,1
79304,1053622035,1
79347,1538703194,1
79348,1881770485,1


In [151]:
#functions for informational purpose
def count_null(df):
    null_counts_per_column = []

    for column in df.columns:
        null_count = df[column].isnull().sum()
        null_counts_per_column.append((column, null_count))

    return null_counts_per_column

def count_zero(df, column):
    print(df[df[column] == 0].shape, df[df[column] != 0].shape)

In [152]:
# Filter df_leie where 'Nearest Year' is greater than or equal to year_of_df
filtered_leie = df_leie[df_leie['fraud'] == 1]

# Create a set of NPI numbers from the filtered df_leie
unique_npi_leie_filtered = set(filtered_leie['NPI'])

df_final['fraud'] = 0
df_final['fraud'] = df_final['Rndrng_NPI'].isin(unique_npi_leie_filtered).astype(int)

df_final[df_final['fraud'] == 0] # non-fraudulent labeled instances
df_final[df_final['fraud'] == 1] # fraudulent labeled instances
# len(df_prov_svc_1[df_prov_svc_1['fraud'] == 1]['NPI'].unique())

# merged_df = pd.merge(df_prov_svc, df_leie_1, left_on='Rndrng_NPI', right_on='NPI')

# distinct_npis = merged_df['NPI'].unique()

# print(len(distinct_npis))

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Pymt_Amt,fraud
0,1003000126,M,Internal Medicine,116,191.0,191,125.000000,83.908220,0
1,1003000126,M,Internal Medicine,40,47.0,47,173.829787,118.570638,0
2,1003000126,M,Internal Medicine,38,39.0,39,257.620513,61.066923,0
3,1003000126,M,Internal Medicine,21,21.0,21,1192.656191,141.442857,0
4,1003000126,M,Internal Medicine,12,12.0,12,319.666667,105.700833,0
...,...,...,...,...,...,...,...,...,...
9886172,1992999825,M,Otolaryngology,114,152.0,152,291.000000,104.546513,0
9886173,1992999874,F,Internal Medicine,51,51.0,51,699.117647,163.101961,0
9886174,1992999874,F,Internal Medicine,114,259.0,259,249.467181,59.407220,0
9886175,1992999874,F,Internal Medicine,255,606.0,606,355.699670,83.646320,0


,Rndrng_NPI,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Pymt_Amt,fraud
35659,1003278615,F,Nurse Practitioner,77,217.0,217,195.8100,112.811106,1
35660,1003278615,F,Nurse Practitioner,103,260.0,260,180.5700,101.642423,1
35661,1003278615,F,Nurse Practitioner,14,20.0,20,242.5845,187.805500,1
90267,1003926270,M,Physical Medicine and Rehabilitation,13,16.0,16,70.0000,30.843750,1
90268,1003926270,M,Physical Medicine and Rehabilitation,19,21.0,21,101.0000,49.081905,1
...,...,...,...,...,...,...,...,...,...
9860815,1992814628,M,Plastic and Reconstructive Surgery,15,17.0,15,2015.0000,556.276471,1
9860816,1992814628,M,Plastic and Reconstructive Surgery,70,70.0,70,222.0000,76.917286,1
9860817,1992814628,M,Plastic and Reconstructive Surgery,69,115.0,115,148.0000,60.440087,1
9865837,1992860977,M,Internal Medicine,29,56.0,56,10.0000,3.000000,1


In [153]:
# Specify the columns you want to one-hot encode
columns_to_encode = ['Rndrng_Prvdr_Type', 'Rndrng_Prvdr_Gndr']

# Perform one-hot encoding
df_final = pd.get_dummies(df_final, columns=columns_to_encode)

# Print the encoded DataFrame


In [154]:
df_final[df_final['fraud']==1]
df_final[df_final['fraud']==0]
df_final.shape

,Rndrng_NPI,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Pymt_Amt,fraud,Rndrng_Prvdr_Type_Addiction Medicine,Rndrng_Prvdr_Type_Adult Congenital Heart Disease,Rndrng_Prvdr_Type_Advanced Heart Failure and Transplant Cardiology,...,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M
35659,1003278615,77,217.0,217,195.8100,112.811106,1,False,False,False,...,False,False,False,False,False,False,False,False,True,False
35660,1003278615,103,260.0,260,180.5700,101.642423,1,False,False,False,...,False,False,False,False,False,False,False,False,True,False
35661,1003278615,14,20.0,20,242.5845,187.805500,1,False,False,False,...,False,False,False,False,False,False,False,False,True,False
90267,1003926270,13,16.0,16,70.0000,30.843750,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
90268,1003926270,19,21.0,21,101.0000,49.081905,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9860815,1992814628,15,17.0,15,2015.0000,556.276471,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9860816,1992814628,70,70.0,70,222.0000,76.917286,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9860817,1992814628,69,115.0,115,148.0000,60.440087,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9865837,1992860977,29,56.0,56,10.0000,3.000000,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True


,Rndrng_NPI,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Pymt_Amt,fraud,Rndrng_Prvdr_Type_Addiction Medicine,Rndrng_Prvdr_Type_Adult Congenital Heart Disease,Rndrng_Prvdr_Type_Advanced Heart Failure and Transplant Cardiology,...,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M
0,1003000126,116,191.0,191,125.000000,83.908220,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1003000126,40,47.0,47,173.829787,118.570638,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1003000126,38,39.0,39,257.620513,61.066923,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1003000126,21,21.0,21,1192.656191,141.442857,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1003000126,12,12.0,12,319.666667,105.700833,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9886172,1992999825,114,152.0,152,291.000000,104.546513,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9886173,1992999874,51,51.0,51,699.117647,163.101961,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9886174,1992999874,114,259.0,259,249.467181,59.407220,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9886175,1992999874,255,606.0,606,355.699670,83.646320,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False


(9288371, 102)

In [161]:
agg_funcs = {
    'Tot_Benes': ['mean', 'sum', 'median', 'std', 'min', 'max'], 
    'Tot_Srvcs': ['mean', 'sum', 'median', 'std', 'min', 'max'],
    'Tot_Bene_Day_Srvcs': ['mean', 'sum', 'median', 'std', 'min', 'max'],
    'Avg_Sbmtd_Chrg': ['mean', 'sum', 'median', 'std', 'min', 'max'],
    'Avg_Mdcr_Pymt_Amt': ['mean', 'sum', 'median', 'std', 'min', 'max']
}

df_aggregated_2 = df_final.groupby('Rndrng_NPI').agg(agg_funcs).reset_index()
df_aggregated_2.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_aggregated_2.columns]

# Print the aggregated DataFrame
scaler = MinMaxScaler()

# Select columns to scale, excluding 'Rndrng_NPI'
numeric_columns = df_aggregated_2.columns.difference(['Rndrng_NPI'])

# Apply Min-Max scaling
df_aggregated_2[numeric_columns] = (df_aggregated_2[numeric_columns] - df_aggregated_2[numeric_columns].min()) / (df_aggregated_2[numeric_columns].max() - df_aggregated_2[numeric_columns].min())

print(df_aggregated_2)

         Rndrng_NPI  Tot_Benes_mean  Tot_Benes_sum  Tot_Benes_median  \
0        1003000126        0.000582       0.002006          0.000228   
1        1003000134        0.002695       0.007229          0.000574   
2        1003000142        0.000192       0.000892          0.000067   
3        1003000423        0.000147       0.000125          0.000071   
4        1003000480        0.000080       0.000120          0.000067   
...             ...             ...            ...               ...   
1059397  1992999551        0.000490       0.000914          0.000196   
1059398  1992999569        0.000034       0.000007          0.000031   
1059399  1992999585        0.000042       0.000008          0.000039   
1059400  1992999825        0.000332       0.000747          0.000259   
1059401  1992999874        0.001141       0.000969          0.001006   

         Tot_Benes_std  Tot_Benes_min  Tot_Benes_max  Tot_Srvcs_mean  \
0             0.001092       0.000026       0.001175        0.0

In [156]:
df_aggregated_2.shape
df_aggregated_2.columns

(1059402, 31)

Index(['Rndrng_NPI', 'Tot_Benes_mean', 'Tot_Benes_sum', 'Tot_Benes_median',
       'Tot_Benes_std', 'Tot_Benes_min', 'Tot_Benes_max', 'Tot_Srvcs_mean',
       'Tot_Srvcs_sum', 'Tot_Srvcs_median', 'Tot_Srvcs_std', 'Tot_Srvcs_min',
       'Tot_Srvcs_max', 'Tot_Bene_Day_Srvcs_mean', 'Tot_Bene_Day_Srvcs_sum',
       'Tot_Bene_Day_Srvcs_median', 'Tot_Bene_Day_Srvcs_std',
       'Tot_Bene_Day_Srvcs_min', 'Tot_Bene_Day_Srvcs_max',
       'Avg_Sbmtd_Chrg_mean', 'Avg_Sbmtd_Chrg_sum', 'Avg_Sbmtd_Chrg_median',
       'Avg_Sbmtd_Chrg_std', 'Avg_Sbmtd_Chrg_min', 'Avg_Sbmtd_Chrg_max',
       'Avg_Mdcr_Pymt_Amt_mean', 'Avg_Mdcr_Pymt_Amt_sum',
       'Avg_Mdcr_Pymt_Amt_median', 'Avg_Mdcr_Pymt_Amt_std',
       'Avg_Mdcr_Pymt_Amt_min', 'Avg_Mdcr_Pymt_Amt_max'],
      dtype='object')

In [157]:
df_encoded_1 = df_final.drop(columns=['Tot_Benes', 'Tot_Srvcs', 'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Pymt_Amt'])
df_aggregated_1 = df_encoded_1.groupby('Rndrng_NPI').sum().reset_index()

# Print the aggregated DataFrame
df_aggregated_1.shape
df_aggregated_1[df_aggregated_1['fraud']>0]

(1059402, 97)

,Rndrng_NPI,fraud,Rndrng_Prvdr_Type_Addiction Medicine,Rndrng_Prvdr_Type_Adult Congenital Heart Disease,Rndrng_Prvdr_Type_Advanced Heart Failure and Transplant Cardiology,Rndrng_Prvdr_Type_Allergy/ Immunology,Rndrng_Prvdr_Type_Ambulance Service Provider,Rndrng_Prvdr_Type_Anesthesiology,Rndrng_Prvdr_Type_Anesthesiology Assistant,Rndrng_Prvdr_Type_Audiologist,...,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M
3983,1003278615,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
9649,1003926270,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
9804,1003939471,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
22680,1023045606,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24003,1023087293,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053848,1992723282,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1054193,1992735591,35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35
1056414,1992799704,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22
1056726,1992814628,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [158]:
# Assuming df_aggregated_1 and df_aggregated_2 are your DataFrames

# Merge the two DataFrames on 'NPI' column
merged_df = pd.merge(df_aggregated_2, df_aggregated_1, on='Rndrng_NPI')

# Print the merged DataFrame
merged_df.loc[merged_df['fraud'] > 0, 'fraud'] = 1
merged_df[merged_df['fraud']==1]
merged_df

,Rndrng_NPI,Tot_Benes_mean,Tot_Benes_sum,Tot_Benes_median,Tot_Benes_std,Tot_Benes_min,Tot_Benes_max,Tot_Srvcs_mean,Tot_Srvcs_sum,Tot_Srvcs_median,...,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M
3983,1003278615,0.000454,0.000309,0.000519,0.000702,0.000077,0.000554,0.000800,0.000244,0.001625,...,0,0,0,0,0,0,0,0,3,0
9649,1003926270,0.000042,0.000035,0.000039,0.000065,0.000052,0.000048,0.000043,0.000014,0.000066,...,0,0,0,0,0,0,0,0,0,2
9804,1003939471,0.000247,0.000593,0.000126,0.000330,0.000180,0.000392,0.000290,0.000296,0.000329,...,0,0,0,0,0,0,0,0,0,9
22680,1023045606,0.000059,0.000012,0.000055,NaN,0.000180,0.000042,0.000041,0.000004,0.000062,...,0,0,0,0,0,0,0,0,0,1
24003,1023087293,0.000509,0.000583,0.000479,0.000570,0.000412,0.000711,0.000334,0.000183,0.000502,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053848,1992723282,0.000017,0.000025,0.000016,0.000000,0.000052,0.000012,0.000653,0.000133,0.000997,...,0,0,0,0,0,0,0,0,0,2
1054193,1992735591,0.002327,0.016923,0.002145,0.003328,0.000052,0.002971,0.004325,0.014946,0.007502,...,0,0,0,0,0,0,0,0,0,35
1056414,1992799704,0.001356,0.006356,0.000094,0.008991,0.000000,0.016691,0.001141,0.002556,0.000148,...,0,0,0,0,0,0,0,0,0,22
1056726,1992814628,0.000206,0.000340,0.000083,0.000408,0.000103,0.000356,0.000173,0.000126,0.000090,...,0,0,0,0,0,0,0,0,0,6


,Rndrng_NPI,Tot_Benes_mean,Tot_Benes_sum,Tot_Benes_median,Tot_Benes_std,Tot_Benes_min,Tot_Benes_max,Tot_Srvcs_mean,Tot_Srvcs_sum,Tot_Srvcs_median,...,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M
0,1003000126,0.000582,0.002006,0.000228,0.001092,0.000026,0.001175,0.001210,0.001842,0.000290,...,0,0,0,0,0,0,0,0,0,15
1,1003000134,0.002695,0.007229,0.000574,0.012892,0.000052,0.018650,0.002835,0.003658,0.000643,...,0,0,0,0,0,0,0,0,0,13
2,1003000142,0.000192,0.000892,0.000067,0.000480,0.000026,0.000645,0.000290,0.000528,0.000137,...,0,0,0,0,0,0,0,0,0,16
3,1003000423,0.000147,0.000125,0.000071,0.000262,0.000155,0.000223,0.000097,0.000039,0.000078,...,0,0,0,0,0,0,0,0,3,0
4,1003000480,0.000080,0.000120,0.000067,0.000120,0.000026,0.000121,0.000059,0.000038,0.000090,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059397,1992999551,0.000490,0.000914,0.000196,0.001156,0.000052,0.001151,0.000493,0.000421,0.000203,...,0,0,0,0,0,0,0,0,8,0
1059398,1992999569,0.000034,0.000007,0.000031,NaN,0.000103,0.000024,0.000025,0.000002,0.000038,...,0,0,0,0,0,0,0,0,1,0
1059399,1992999585,0.000042,0.000008,0.000039,NaN,0.000129,0.000030,0.000252,0.000025,0.000384,...,0,0,0,0,0,0,0,0,0,1
1059400,1992999825,0.000332,0.000747,0.000259,0.000496,0.000000,0.000621,0.000254,0.000264,0.000321,...,0,0,0,0,0,0,0,0,0,9


In [159]:
merged_df.to_csv(year_of_df+".csv", index=False)

In [160]:
#to check if aggregation took place and we don't have reapeating NPI numbers
len(merged_df['Rndrng_NPI'].unique())

1059402